In [ ]:
%matplotlib inline
import html
from collections import defaultdict
from pathlib import Path

from ipywidgets import widgets

import pandas as pd
import matplotlib as mpl
import compress_pickle

from hpc_inspect.inspector import *

In [ ]:
data = compress_pickle.load('dataframe.pickle.lzma')
#data.set_index(['pmodel', 'compiler', 'incore_model', 'cores', 'define'], inplace=True)
compilers = [cc for cc in data.compiler.unique() if cc is not None]
incore_models = [icm for icm in data.incore_model.unique() if icm is not None]
cache_predictors = [cp for cp in data.cache_predictor]

In [ ]:
data.columns

## Stencil Properties

In [ ]:
# TODO present origin and property of kernel code
iterations_per_cacheline = int(data.get('iterations per cacheline').dropna().unique()[0])

## Kernel Source Code

In [ ]:
print(data.iloc[0].job.workload.kernel.get_code())

### In-Core Analysis

In [ ]:
# Get analysis alongside with assembly for each compiler and in-core model
cc_tab = widgets.Tab()
cc_tab_children = []
for i, cc in enumerate(compilers):
    cc_tab.set_title(i, cc)
    icm_tab = widgets.Tab(children=[])
    icm_tab_children = []
    cc_tab_children.append(icm_tab)
    for j, icm in enumerate(incore_models):
        icm_tab.set_title(j, icm)
        icm_tab_children.append(
            widgets.HTML(value='<pre style="line-height: 1;">{}</pre>'.format(html.escape(
                data.query("compiler == @cc and incore_model == @icm")['in-core model output'].unique()[0]))))
    icm_tab.children = icm_tab_children
cc_tab.children = cc_tab_children

cc_tab

## Layer Conditions

In [ ]:
# TODO readout and present LCs
data.columns

## Single Core Grid Scaling

### ECM Prediction vs Performance

In [ ]:
data_defines = data.sort_values(by=['define'])
fig, axs = mpl.pyplot.subplots(len(compilers), len(incore_models), squeeze=False,
                               figsize=(7*len(incore_models),4*len(compilers)))
for i_cc, cc in enumerate(compilers):
    for i_icm, icm in enumerate(incore_models):
        ax = axs[i_cc, i_icm]
        ax.set_title("{} {}".format(cc, icm))
        ax.set_ylabel("cycle per {} iterations".format(iterations_per_cacheline))
        ax.set_xlabel("")
        ecm_data = data_defines.query('pmodel=="ECM" and cores==1 and incore_model == @icm')
        ecm_data.plot.area(
            y=['T_RegL1', 'T_L1L2', 'T_L2L3', 'T_L3MEM'], x='define', logx=True, xlim=100, ylim=0, ax=ax)
        data_defines.query('pmodel=="Benchmark" and cores==1').plot(
            y='performance [cy/CL]', x='define', logx=True, xlim=100, ylim=0, label='Measured', ax=ax)
        data_defines.query('pmodel=="RooflineIACA" and incore_model == @icm').plot(
            y='performance [cy/CL]', x='define', logx=True, xlim=100, ylim=0, label='RL pred.', ax=ax)
# TODO improve legend (and reduce to one)
# TODO name columns and rows
# TODO fix scale/limits to show all available data
# TODO add It/s, flop/s axis to right
# TODO find good metric (or name) for x-axis

### Data Transfers

In [ ]:
for cp in cache_predictors:
    data.query('cache_predictor==@cp')

# TODO include predicted information into pandas
# TODO include measured informatin into pandas (inspector.py:574)

## Multi Core Thread Sacling

In [ ]:
data_cores = data.sort_values(by=['cores'])
fig, axs = mpl.pyplot.subplots(len(compilers), len(incore_models), squeeze=False,
                               figsize=(7*len(incore_models),4*len(compilers)))
max_define = data.define.max()
for i_cc, cc in enumerate(compilers):
    for i_icm, icm in enumerate(incore_models):
        ax = axs[i_cc, i_icm]
        ax.set_title("{} {}".format(cc, icm))
        data_cores.query('pmodel=="Benchmark" and define==@max_define').plot(
            x='cores', y='performance [cy/CL]', xlim=1, ylim=0, legend=True, label='Measured', ax=ax)
        data_cores.query('pmodel=="ECM" and define==@max_define and incore_model==@icm').plot(
            x='cores', y='performance [cy/CL]', xlim=1, ylim=0, legend=True, label='ECM pred.', ax=ax)
        data_cores.query('pmodel=="RooflineIACA" and define==@max_define and incore_model==@icm').plot(
            x='cores', y='performance [cy/CL]', xlim=1, ylim=0, legend=True, label='RL pred.', ax=ax)
        ax.set_xticks(range(int(data.cores.min()), int(data.cores.max() + 1)))
# TODO find issue with ECM scaling curve

## System Information

In [ ]:
# TODO collect and present machine state

# Notebook things to look at:
 * https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Styling.html
